# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data_df = pd.read_csv("../output_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,rikitea,-23.1203,-134.9692,76.84,78,18,19.91,PF,1614456980
1,1,albany,42.6001,-73.9662,44.01,96,100,1.01,US,1614456969
2,2,saskylakh,71.9167,114.0833,-27.83,76,27,2.51,RU,1614456980
3,3,mar del plata,-38.0023,-57.5575,73.99,88,20,19.57,AR,1614456962
4,4,coos bay,43.3665,-124.2179,50.00,62,90,9.22,US,1614456721


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
narrowed_city_df = city_data_df.loc[(city_data_df["Max Temp"] < 80) & (city_data_df["Max Temp"] > 70) \
                                    & (city_data_df["Wind Speed"] < 10) \
                                    & (city_data_df["Cloudiness"] == 0)].dropna()
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
20,20,noumea,-22.2763,166.4572,79.00,78,0,4.61,NC,1614456718
23,23,dingle,10.9995,122.6711,75.87,89,0,5.46,PH,1614456983
51,51,ushuaia,-54.8000,-68.3000,75.20,33,0,6.91,AR,1614456989
99,99,pagsangahan,13.2158,122.5431,78.62,84,0,6.46,PH,1614457000
109,109,salalah,17.0151,54.0924,77.00,69,0,2.30,OM,1614457002
136,136,pisco,-13.7000,-76.2167,77.00,69,0,6.91,PE,1614457008
200,200,bonthe,7.5264,-12.5050,79.63,82,0,7.94,SL,1614457021
208,208,dwarka,22.2394,68.9678,74.21,89,0,9.04,IN,1614457022
224,224,port blair,11.6667,92.7500,75.42,73,0,4.85,IN,1614457025
241,241,vredendal,-31.6683,18.5012,70.57,55,0,8.59,ZA,1614457028


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = narrowed_city_df

params = {"radius": 5000, "types": "lodging", "key": g_key}


for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]

   
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
  
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
  

Retrieving Results for Index 20: noumea.
Closest hotel is Hôtel Le Lagon.
Retrieving Results for Index 23: dingle.
Closest hotel is SEGAYA PROPERTY.
Retrieving Results for Index 51: ushuaia.
Closest hotel is Albatros Hotel.
Retrieving Results for Index 99: pagsangahan.
Closest hotel is Paquibo Rosas Bahay.
Retrieving Results for Index 109: salalah.
Closest hotel is HAMDAN PLAZA HOTEL SALALAH.
Retrieving Results for Index 136: pisco.
Closest hotel is Hotel San Isidro Oficial.
Retrieving Results for Index 200: bonthe.
Closest hotel is Bonthe Holiday Village.
Retrieving Results for Index 208: dwarka.
Closest hotel is Hotel Narayan Avezika Comfort.
Retrieving Results for Index 224: port blair.
Closest hotel is Welcomhotel Bay Island Port Blair.
Retrieving Results for Index 241: vredendal.
Closest hotel is Tharrakamma Guest House.
Retrieving Results for Index 268: banda aceh.
Closest hotel is Sulthan Hotel International.
Retrieving Results for Index 339: rayachoti.
Closest hotel is Swetha l

In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map

hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info)

# Display figure
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
#fig.add_layer(markers)
#fig.add_layer(hotel_layer)
fig


Figure(layout=FigureLayout(height='420px'))